In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


from matplotlib import cm
from matplotlib.colors import ListedColormap, LinearSegmentedColormap

N = 256
vals = np.ones((N, 4))
vals[:, 0] = np.linspace(255/256, 1, N)    ## red  255,192, 203
vals[:, 1] = np.linspace(192/256, 1, N)
vals[:, 2] = np.linspace(203/256, 1, N)  ## blue 75, 0, 130
Pinks = ListedColormap(vals)


top = cm.get_cmap(Pinks, 256)
bottom = cm.get_cmap('Purples', 256)

newcolors = np.vstack((top(np.linspace(0, 1, 256)),
                       bottom(np.linspace(0, 1, 256))))

newcmp = ListedColormap(newcolors, name='PinkPueple')

In [ ]:
guests_characteristics = pd.read_csv('../data/Predykcyjne_guest_characteristics_.csv')
guests_cusine =  pd.read_csv('../data/Predykcyjne_guest_cuisine_.csv')
guests_payment =  pd.read_csv('../data/Predykcyjne_guest_payment_.csv')

guests_characteristics.drop( 'Unnamed: 0', axis = 1, inplace = True)
guests_cusine.drop( 'Unnamed: 0', axis = 1, inplace = True)
guests_payment.drop( 'Unnamed: 0', axis = 1, inplace = True)

restaurant_cusine =pd.read_csv('../data/Predykcyjne_restaurant_cuisine.csv')
restaurant_open =pd.read_csv('../data/Predykcyjne_restaurant_open.csv')
restaurant_parking =pd.read_csv('../data/Predykcyjne_restaurant_parking.csv')
restaurant_payment =pd.read_csv('../data/Predykcyjne_restaurant_payment.csv')

stars = pd.read_csv("../data/Predykcyjne_Stars_.csv")

## Guest Table

In [ ]:
df_inner = pd.merge(guests_characteristics, guests_cusine, on='guestID', how='inner')
guests = pd.merge(df_inner, guests_payment, on='guestID', how='inner')
guests

In [ ]:
my_info = [guests['smoking'].value_counts(),
            guests['drinker'].value_counts(),
            guests['dress_preference'].value_counts(),
            guests['company'].value_counts(),
            guests['transport'].value_counts(),
            guests['marital_status'].value_counts(),
            guests['interest'].value_counts(),
            guests['religion'].value_counts(),
            guests['activity'].value_counts(),
            guests['color'].value_counts(),
            guests['budget'].value_counts(),
            guests['cuisine'].value_counts(),
            guests['payment'].value_counts()]

#for info in my_info:
#    print(info, "\n\n")

In [ ]:
guests.drop(guests.index[guests['dress_preference'] == '?'], inplace = True)
guests.drop(guests.index[guests['company'] == '?'], inplace = True)
guests.drop(guests.index[guests['transport'] == '?'], inplace = True)
guests.drop(guests.index[guests['marital_status'] == '?'], inplace = True)
guests.drop(guests.index[guests['activity'] == '?'], inplace = True)
guests.drop(guests.index[guests['budget'] == '?'], inplace = True)
guests

## Merge stars with guests

In [ ]:
stars_guests = pd.merge(guests, stars, on='guestID', how='inner')
stars_guests.drop( 'Unnamed: 0', axis = 1, inplace = True)
stars_guests.drop( 'guestID', axis = 1, inplace = True)

stars_guests.drop_duplicates(inplace = True)
stars_guests

In [ ]:
stars_guests['smoking'] = stars_guests['smoking'].astype(object)
stars_guests['drinker'] = stars_guests['drinker'].astype(object)
stars_guests['dress_preference'] = stars_guests['dress_preference'].astype(object)
stars_guests['company'] = stars_guests['company'].astype(object)
stars_guests['transport'] = stars_guests['transport'].astype(object)
stars_guests['marital_status'] = stars_guests['marital_status'].astype(object)
stars_guests['interest'] = stars_guests['interest'].astype(object)
stars_guests['religion'] = stars_guests['religion'].astype(object)
stars_guests['activity'] = stars_guests['activity'].astype(object)
stars_guests['budget'] = stars_guests['budget'].astype(object)
stars_guests['cuisine'] = stars_guests['cuisine'].astype(object)
stars_guests['payment'] = stars_guests['payment'].astype(object)
stars_guests['color'] = stars_guests['color'].astype(object)

# Some of the predictors are categorical. Lets produce some dummy variables
categorical_bolean = (stars_guests.dtypes == "object").values
data_numeric = stars_guests.loc[:, ~categorical_bolean]
data_categorical = stars_guests.loc[:, categorical_bolean]
data_dummies = pd.get_dummies(data_categorical)

data = pd.concat([data_numeric, data_dummies], axis=1)
data['target'] = data['stars']
data.head()

In [ ]:
corr_matrix = np.corrcoef(data.T)
pd.DataFrame(corr_matrix)
plt.figure(figsize=(10,10))
plt.imshow(corr_matrix, cmap = newcmp, vmax = .5, vmin = -.5)
plt.colorbar()
plt.show()

In [ ]:
n = len(corr_matrix[-1])
t = np.linspace(0,n-1, n)
plt.scatter(t, corr_matrix[-1], s=3, c='indigo')
plt.hlines(y=0.005, xmin=0, xmax = 108, color = 'gray', alpha = 0.5)
plt.hlines(y=-0.005, xmin=0, xmax = 108, color = 'gray', alpha = 0.5)
plt.ylim(-0.03, 0.03)

In [ ]:
indexes = list(np.where( abs(corr_matrix[-1]) < 0.005)[0])

print("list of {:.2f} % columns that are weakly (corr < 0.05) corellated with target\n\n".format( len(indexes)/n * 100))
for name in data.columns[indexes]:
    print(name)

In [ ]:
indexes = list(np.where( abs(corr_matrix[-1]) >0.07)[0])

print("list of {:.2f} % columns that are strongly (corr > 0.07) corellated with target\n\n".format( len(indexes)/n * 100))
for name in data.columns[indexes]:
    print(name)

## Cash table

In [ ]:
cash = pd.DataFrame(data['target'])
cash['payment_cash'] = data['payment_cash']
cash['payment_American_Express'] = data['payment_American_Express']
cash['payment_MasterCard-Eurocard'] = data['payment_MasterCard-Eurocard']
cash['payment_VISA'] = data['payment_VISA']
cash['payment_bank_debit_cards'] = data['payment_bank_debit_cards']
cash['budget_high'] = data['budget_high']
cash['budget_low'] = data['budget_low']
cash['budget_medium'] = data['budget_medium']

cash

In [ ]:
corr_matrix = np.corrcoef(cash.T)
pd.DataFrame(corr_matrix)
plt.figure(figsize=(10,10))
plt.imshow(corr_matrix, cmap = newcmp, vmax = 0.2, vmin = -.2)
plt.colorbar()
plt.yticks([0,1,2,3,4,5,6,7,8])
plt.xticks([0,1,2,3,4,5,6,7,8])
plt.show()

## Restaurant Table

In [ ]:
df_inner = pd.merge(restaurant_cusine, restaurant_open, on='restaurantID', how='inner')
df_inner = pd.merge(df_inner, restaurant_parking, on='restaurantID', how='inner')
restaurant = pd.merge(df_inner, restaurant_payment, on='restaurantID', how='inner')

In [ ]:
restaurant['days;;;;;;;;'].value_counts()

In [ ]:
restaurant['hours'].value_counts()

In [ ]:
restaurant.replace("Mon;Tue;Wed;Thu;Fri;;;", "Workdays", inplace=True)
restaurant.replace("Mon;Tue;Wed;Thu;Fri;;", "Workdays", inplace=True)
restaurant.replace("Mon;Tue;Wed;Thu;Fri", "Workdays",inplace=True)

restaurant.replace("Sat;;;;;;;", "Sat",inplace=True)
restaurant.replace("Sat;;;;;;", "Sat",inplace=True)
restaurant.replace("Sat;;;;;", "Sat",inplace=True)
restaurant.replace("Sat;;;;", "Sat",inplace=True)

restaurant.replace("Sun;;;;;;;", "Sun",inplace=True)
restaurant.replace("Sun;;;;;;", "Sun", inplace=True)
restaurant.replace("Sun;;;;;", "Sun",inplace=True)
restaurant.replace("Sun;;;;", "Sun",inplace=True)

restaurant.rename(columns={"days;;;;;;;;" : "days"}, inplace=True)
restaurant['days'].value_counts()

In [ ]:
n = len(restaurant['hours'])
opening = []
closing = []
long = []
for i in range(n):
    o = restaurant['hours'][i].split( "-" )[0]
    o = int(o[:2])
    c = restaurant['hours'][i].replace(";", "-").split( "-" )[1]
    c = int(c[:2])
    if c == 0:
        c = 24
    opening.append(o)
    closing.append(c)
    
restaurant['opening'] = opening
restaurant['closing'] = closing
restaurant.head(20)

In [ ]:
def get_open_marker(start, end, m):
    
    tmp = np.zeros(m, dtype = int)
    
    for j in range(m):
        if restaurant['opening'][j] ==0 and restaurant['closing'][j] ==0:
            tmp[j]=3
            
        elif restaurant['opening'][j] < restaurant['closing'][j]:
            if restaurant['opening'][j] <= start and  restaurant['closing'][j] >= end:
                tmp[j]=1
            
        elif restaurant['opening'][j] > restaurant['closing'][i]:
            if restaurant['closing'][j] <= start and restaurant['opening'][j]>= end:
                tmp[j]=0
            else:
                tmp[j]=1
      
    
    return tmp
            

In [ ]:
m = len(restaurant['hours'])
restaurant['open_00_01'] = get_open_marker(0, 1, m)
restaurant['open_01_02'] = get_open_marker(1, 2, m)
restaurant['open_02_03'] = get_open_marker(2, 3, m)
restaurant['open_03_04'] = get_open_marker(3, 4, m)
restaurant['open_04_05'] = get_open_marker(4, 5, m)
restaurant['open_05_06'] = get_open_marker(5, 6, m)
restaurant['open_06_07'] = get_open_marker(6, 7, m)
restaurant['open_07_08'] = get_open_marker(7, 8, m)
restaurant['open_08_09'] = get_open_marker(8, 9, m)
restaurant['open_09_10'] = get_open_marker(9, 10, m)
restaurant['open_10_11'] = get_open_marker(10, 11, m)
restaurant['open_11_12'] = get_open_marker(11, 12, m)
restaurant['open_12_13'] = get_open_marker(12, 13, m)
restaurant['open_13_14'] = get_open_marker(13, 14, m)
restaurant['open_14_15'] = get_open_marker(14, 15, m)
restaurant['open_15_16'] = get_open_marker(15, 16, m)
restaurant['open_16_17'] = get_open_marker(16, 17, m)
restaurant['open_17_18'] = get_open_marker(17, 18, m)
restaurant['open_18_19'] = get_open_marker(18, 19, m)
restaurant['open_19_20'] = get_open_marker(19, 20, m)
restaurant['open_20_21'] = get_open_marker(20, 21, m)
restaurant['open_21_22'] = get_open_marker(21, 22, m)
restaurant['open_22_23'] = get_open_marker(22, 23, m)
restaurant['open_23_24'] = get_open_marker(23, 24, m)

In [ ]:
restaurant.head(10)

In [ ]:
stars_restaurant = pd.merge(restaurant, stars, on='restaurantID', how='inner')
stars_restaurant.drop('hours', axis = 1, inplace = True)
stars_restaurant.drop('Unnamed: 0', axis = 1, inplace = True)
stars_restaurant.drop('guestID', axis = 1, inplace = True)

stars_restaurant

In [ ]:
stars_restaurant['cuisine'] = stars_restaurant['cuisine'].astype(object)
stars_restaurant['days'] = stars_restaurant['days'].astype(object)
stars_restaurant['parking_lot'] = stars_restaurant['parking_lot'].astype(object)
stars_restaurant['payment'] = stars_restaurant['payment'].astype(object)


# Some of the predictors are categorical. Lets produce some dummy variables
categorical_bolean = (stars_restaurant.dtypes == "object").values
data_numeric = stars_restaurant.loc[:, ~categorical_bolean]
data_categorical = stars_restaurant.loc[:, categorical_bolean]
data_dummies = pd.get_dummies(data_categorical)

data = pd.concat([data_numeric, data_dummies], axis=1)
data['target'] = data['stars']
data.head()

In [ ]:
corr_matrix = np.corrcoef(data.T)
pd.DataFrame(corr_matrix)
plt.figure(figsize=(10,10))
plt.imshow(corr_matrix, cmap = newcmp, vmax = .5, vmin = -.5)
plt.colorbar()
plt.show()

In [ ]:
n = len(corr_matrix[-1])
t = np.linspace(0,n-1, n)
plt.scatter(t, corr_matrix[-1], s=3, c='indigo')
plt.hlines(y=0.01, xmin=0, xmax = 70, color = 'gray', alpha = 0.5)
plt.hlines(y=-0.01, xmin=0, xmax = 70, color = 'gray', alpha = 0.5)
plt.ylim(-0.1, 0.1)


In [ ]:
indexes = list(np.where( abs(corr_matrix[-1]) < 0.005)[0])

print("list of {:.2f} % columns that are weakly (corr < 0.005) corellated with target\n\n".format( len(indexes)/n * 100))
for name in data.columns[indexes]:
    print(name)

In [ ]:
indexes = list(np.where( abs(corr_matrix[-1]) >0.01)[0])

print("list of {:.2f} % columns that are strongly (corr > 0.01) corellated with target\n\n".format( len(indexes)/n * 100))
for name in data.columns[indexes]:
    print(name)

In [ ]:
indexes = list(np.where( abs(corr_matrix[-1]) >0.05)[0])

print("list of {:.2f} % columns that are very strongly (corr > 0.05) corellated with target\n\n".format( len(indexes)/n * 100))
for name in data.columns[indexes]:
    print(name)

# Stars table

In [ ]:
stars.drop( 'Unnamed: 0', axis = 1, inplace = True)
guestID = []
for i in range(len(stars)):
    guestID.append(int(stars['guestID'][i][1:]))
stars['guestID'] = guestID

In [ ]:
corr_matrix = np.corrcoef(stars.T)
pd.DataFrame(corr_matrix)
plt.figure(figsize=(10,10))
plt.imshow(corr_matrix, cmap = newcmp, vmax = 1, vmin = -1)
plt.colorbar()
plt.show()

## Comment:

Based on the correlation matrices for guests, restaurants and stars I decided to remove from future data sets columns: 
* "opening" and "days" in restaurants data frame (reason: small correlation with target) 
* "payment" (reason:  we have also acces to how clients pay for dinner, so as long we know, that the client managed to pay, we dont need to know, which other methods were avaiable). 
* "guestID", "restaurantID", "stars" (they are practically independent)

I also replace "birth_year" with "age" calculated as age = (2021 - birth_year)
